In [4]:
%cd ../
%reload_ext autoreload
%autoreload 2

/home/akali/projects/model-trainer


In [20]:
import importlib
import json
import logging

from tqdm import tqdm

from model_trainer import loader
from model_trainer.config import get_path

log = logging.getLogger(__name__)
importlib.reload(loader)

<module 'model_trainer.loader' from '/projects/model-trainer/model_trainer/loader.py'>

In [9]:
corpus_name = "wiki-he-filtered"

In [10]:
w2v_cbow_path = get_path(corpus_name, "cbow.kv")
w2v_cbow_model = loader.load_kv(w2v_cbow_path)

# w2v_sg_path = get_path(corpus_name, "sg.kv")
# w2v_cbow = loader.load_kv(w2v_cbow_path)

# ft_path = get_path(corpus_name, "fasttext.kv")
# ft_model = loader.load_kv(ft_path)

NameError: name 'get_path' is not defined

In [10]:
w2v_cbow_vocab = set(w2v_cbow_model.key_to_index.keys())
ft_vocab = w2v_cbow_vocab  # set(ft_model.key_to_index.keys())
common_vocab = w2v_cbow_vocab.intersection(ft_vocab)

s1, s2 = len(w2v_cbow_vocab), len(ft_vocab)
print(f"Vocabulary diff: {abs(s1 - s2)}")
print(f"Common vocab: {len(common_vocab)}")

Vocabulary diff: 0
Common vocab: 473539


In [13]:
import random
from model_trainer import stemming
from model_trainer.stemming import StemDictGenerator, reduce_stem_dict

importlib.reload(stemming)

stemming_generator = StemDictGenerator(model=w2v_cbow_model)

# random.seed(123)
# words = random.sample(common_vocab, k=1000)

all_stemming = stemming_generator.generate_model_stemming()

print(all_stemming)
serialized = json.dumps(all_stemming)
with open("stemming.json", "w") as file:
    file.write(serialized)

100%|██████████| 473539/473539 [4:15:20<00:00, 30.91it/s]  

[04:07:50] Total 26156 inflections generated [model_trainer.inflections.inflections_dict_generator]
[04:07:50] Function 'generate_model_inflections' took 4:15:20.237025. [utils.logging]
[04:07:50] Reducing dict of size 26156 [model_trainer.inflections.inflections_dict_reducer]



100%|██████████| 26156/26156 [00:00<00:00, 107027.32it/s]

[04:07:50] Function 'reduce_inflections_dict' took 0:00:00.268027. [utils.logging]
{'ובשנת': 'וב', 'שהיה': 'היה', 'ולאחר': 'לאחר', 'שלא': 'לא', 'ולא': 'לא', 'בידי': 'ידי', 'כשבינ': 'בינ', 'שבינ': 'בינ', 'בינו': 'בינ', 'בינמ': 'בינ', 'כזה': 'זה', 'שכזה': 'זה', 'ועד': 'עד', 'ובית': 'בית', 'בבית': 'בית', 'כבית': 'בית', 'ובבית': 'בית', 'מבית': 'בית', 'לבית': 'בית', 'בקישורימ': 'קישורימ', 'לכל': 'כל', 'בכל': 'כל', 'וכל': 'כל', 'והייתה': 'הייתה', 'שהייתה': 'הייתה', 'הבריתה': 'הברית', 'הבריתעל': 'הברית', 'ואשר': 'אשר', 'כאשר': 'אשר', 'כזו': 'זו', 'והיו': 'היו', 'שהיו': 'היו', 'וכמו': 'כמו', 'שאחד': 'אחד', 'כשאחד': 'אחד', 'וכאשר': 'אשר', 'שבו': 'בו', 'למספר': 'מספר', 'ומספר': 'מספר', 'ובמהלכ': 'במהלכ', 'וכדי': 'כדי', 'בכדי': 'כדי', 'ובנ': 'בנ', 'כבנ': 'בנ', 'מבנ': 'בנ', 'שאותו': 'אותו', 'ויש': 'יש', 'שיש': 'יש', 'ישנמ': 'יש', 'ישנו': 'יש', 'ישנה': 'יש', 'ושארצות': 'שארצות', 'כשארצות': 'שארצות', 'דארצות': 'ארצות', 'שלארצות': 'ארצות', 'כארצות': 'ארצות', 'שבארצות': 'ארצות', 'כשבארצות': 'בארצות', 

[04:07:50] Generating inflections... [model_trainer.inflections.inflections_dict_generator]


100%|██████████| 473539/473539 [4:05:54<00:00, 32.09it/s]  

[08:13:45] Total 26156 inflections generated [model_trainer.inflections.inflections_dict_generator]
[08:13:45] Function 'generate_model_inflections' took 4:05:54.545040. [utils.logging]
[08:13:45] Reducing dict of size 26156 [model_trainer.inflections.inflections_dict_reducer]



100%|██████████| 26156/26156 [00:00<00:00, 383361.63it/s]

[08:13:45] Function 'reduce_inflections_dict' took 0:00:00.111663. [utils.logging]


{'ובשנת': 'וב', 'שהיה': 'היה', 'ולאחר': 'לאחר', 'שלא': 'לא', 'ולא': 'לא', 'בידי': 'ידי', 'כשבינ': 'בינ', 'שבינ': 'בינ', 'בינו': 'בינ', 'בינמ': 'בינ', 'כזה': 'זה', 'שכזה': 'זה', 'ועד': 'עד', 'ובית': 'בית', 'בבית': 'בית', 'כבית': 'בית', 'ובבית': 'בית', 'מבית': 'בית', 'לבית': 'בית', 'בקישורימ': 'קישורימ', 'לכל': 'כל', 'בכל': 'כל', 'וכל': 'כל', 'והייתה': 'הייתה', 'שהייתה': 'הייתה', 'הבריתה': 'הברית', 'הבריתעל': 'הברית', 'ואשר': 'אשר', 'כאשר': 'אשר', 'כזו': 'זו', 'והיו': 'היו', 'שהיו': 'היו', 'וכמו': 'כמו', 'שאחד': 'אחד', 'כשאחד': 'אחד', 'וכאשר': 'אשר', 'שבו': 'בו', 'למספר': 'מספר', 'ומספר': 'מספר', 'ובמהלכ': 'במהלכ', 'וכדי': 'כדי', 'בכדי': 'כדי', 'ובנ': 'בנ', 'כבנ': 'בנ', 'מבנ': 'בנ', 'שאותו': 'אותו', 'ויש': 'יש', 'שיש': 'יש', 'ישנמ': 'יש', 'ישנו': 'יש', 'ישנה': 'יש', 'ושארצות': 'שארצות', 'כשארצות': 'שארצות', 'דארצות': 'ארצות', 'שלארצות': 'ארצות', 'כארצות': 'ארצות', 'שבארצות': 'ארצות', 'כשבארצות': 'בארצות', 'בארצות': 'ארצות', 'כשנה': 'שנה', 'אביבי': 'אביב', 'ואמ': 'אמ', 'ורק': 'רק', 'שבה':

In [8]:
with open("stemming.json") as file:
    reduced_stemming = json.load(file)

print([(key, value) for key, value in reduced_stemming.items() if "מספר" in key])

[('למספר', 'מספר'), ('ומספר', 'מספר'), ('ובמספר', 'במספר'), ('והמספר', 'המספר'), ('מספריו', 'ספריו'), ('המספרימ', 'מספרימ'), ('ומספרת', 'מספרת'), ('שמספרת', 'מספרת'), ('המספרת', 'מספרת'), ('ומספרמ', 'מספרמ'), ('שמספרמ', 'מספרמ'), ('שמספרנ', 'מספרנ'), ('ומספרנ', 'מספרנ')]


In [19]:
original_corpus_path = get_path(corpus_name, "corpus.txt")
reduced_corpus_path = get_path(corpus_name, "corpus-reduced-1.txt")

reduce_corpus(original_corpus_path, reduced_corpus_path)

[22:34:45] Reducing corpus... [__main__]


283717it [00:45, 6173.11it/s]
